In [ ]:
ENTSOE_API_KEY=YOUR_ENTSOE_KEY
OLLAMA_MODEL=llama3


In [ ]:
SUPPORTED_MARKETS = ["day_ahead", "intraday"]

# Pass-through validation (ENTSO-E handles validity)
def validate_inputs(country, market):
    if market not in SUPPORTED_MARKETS:
        raise ValueError("Market must be day_ahead or intraday")
    if len(country) < 2:
        raise ValueError("Invalid country / bidding zone code")


In [ ]:
from entsoe import EntsoePandasClient
import os

client = EntsoePandasClient(api_key=os.getenv("ENTSOE_API_KEY"))

def get_prices(country, market, start, end):
    if market == "day_ahead":
        prices = client.query_day_ahead_prices(country, start, end)
    else:
        prices = client.query_intraday_prices(country, start, end)

    prices = prices.resample("H").mean().dropna()
    return prices


In [ ]:
import pandas as pd
import numpy as np

def get_weather_features(index):
    return pd.DataFrame({
        "temp": np.random.normal(10, 5, len(index)),
        "wind": np.random.normal(8, 3, len(index)),
        "solar": np.random.normal(200, 50, len(index))
    }, index=index)


In [ ]:
import xgboost as xgb
import pandas as pd

class XGBPriceForecaster:
    def __init__(self):
        self.model = xgb.XGBRegressor(
            n_estimators=500,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8
        )

    def _features(self, prices, weather):
        df = pd.DataFrame({"price": prices})
        df["hour"] = df.index.hour
        df["dayofweek"] = df.index.dayofweek
        df = pd.concat([df, weather], axis=1)
        return df.dropna()

    def train(self, prices, weather):
        df = self._features(prices, weather)
        X = df.drop(columns=["price"])
        y = df["price"]
        self.model.fit(X, y)

    def forecast(self, prices, weather, horizon=24):
        df = self._features(prices, weather).iloc[-horizon:]
        X = df.drop(columns=["price"])
        return self.model.predict(X)


In [ ]:
import pyomo.environ as pyo

def optimize_battery(prices, capacity=20, power=10):
    T = range(len(prices))
    m = pyo.ConcreteModel()

    m.c = pyo.Var(T, bounds=(0, power))
    m.d = pyo.Var(T, bounds=(0, power))
    m.soc = pyo.Var(T, bounds=(0, capacity))

    def soc_rule(m, t):
        if t == 0:
            return m.soc[t] == capacity / 2
        return m.soc[t] == m.soc[t-1] + m.c[t] - m.d[t]

    m.soc_c = pyo.Constraint(T, rule=soc_rule)

    m.obj = pyo.Objective(
        expr=sum(prices[t] * (m.d[t] - m.c[t]) for t in T),
        sense=pyo.maximize
    )

    pyo.SolverFactory("glpk").solve(m)

    actions = []
    for t in T:
        if m.c[t].value > 0.1:
            actions.append(f"H{t}: Charge {m.c[t].value:.1f} MW")
        if m.d[t].value > 0.1:
            actions.append(f"H{t}: Discharge {m.d[t].value:.1f} MW")

    return actions


In [ ]:
import subprocess, os

MODEL = os.getenv("OLLAMA_MODEL", "llama3")

def explain(country, market, prices, actions):
    prompt = f"""
You are an expert EPEX Spot trader.

Country: {country}
Market: {market}

Forecast prices:
{prices}

Battery actions:
{actions}

Explain the strategy, risks, and assumptions.
"""
    p = subprocess.Popen(
        ["ollama", "run", MODEL],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        text=True
    )
    out, _ = p.communicate(prompt)
    return out


In [ ]:
import streamlit as st
from datetime import datetime, timedelta

from ingestion.entsoe_client import get_prices
from ingestion.weather_stub import get_weather_features
from models.xgb_forecaster import XGBPriceForecaster
from optimization.battery_optimizer import optimize_battery
from agent.market_reasoner import explain
from config.markets import validate_inputs

st.title("EPEX AI Decision Agent")

country = st.text_input("Bidding Zone (e.g. DE_LU, NL, FR)", "NL")
market = st.selectbox("Market", ["day_ahead", "intraday"])

if st.button("Run Agent"):
    validate_inputs(country, market)

    end = datetime.utcnow()
    start = end - timedelta(days=30)

    prices = get_prices(country, market, start, end)
    weather = get_weather_features(prices.index)

    model = XGBPriceForecaster()
    model.train(prices, weather)
    forecast = model.forecast(prices, weather)

    actions = optimize_battery(forecast)
    reasoning = explain(country, market, forecast.tolist(), actions)

    st.subheader("Forecast Prices")
    st.line_chart(forecast)

    st.subheader("Battery Actions")
    st.write(actions)

    st.subheader("AI Reasoning")
    st.write(reasoning)
